<a href="https://colab.research.google.com/github/colinrsmall/ehm_roster_tools/blob/master/EP_Player_Staff_Cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
api_key = "h3n2KkqNeb4ptLknxTVk1uYuGL8jfLUg" #@param {type:"string"}
most_recent_player_year = "2021" #@param {type:"string"}
most_recent_staff_year = "2020" #@param {type:"string"}

# Expand this if you want to look at the code (optional)

In [5]:
import requests, random, csv, traceback, time, urllib.request
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from google.colab import files

In [53]:
# Read input file
with open(f'/content/input.csv', 'r+', newline='', encoding='UTF-8') as input_file:
  # Set up output file
  with open(f'/content/output.csv', 'w+', newline='', encoding='UTF-8') as output_file:
    output_file.write('\ufeff')
    csvwriter = csv.writer(output_file, delimiter=',')
    top_header = 'staff,dmy,metric,,|--,NATIONALITY,,--|,,|--,,,CLUB CONTRACT,,,,--|,|--,NATION CONTRACT,,--|,|--,NATIONAL TEAM,--|,|-- NHL,--|,|--,EDIT DETAILS,--|,|--,,,STAFF ATTRIBUTES,,,,--|,|-- PLAYER ABILITY,--|,|--,PLAYER REPUTATION,--|,|--,,POSITION,,,--|,|--,ROLE,--|,,|--,JERSEY #,--|,,,,|--,,,,,,MENTAL,,,,,,,--|,|--,,,,PHYSICAL,,,,,--|,|--,,,,,,TECHNICAL,,,,,,,--|,|--,,GOALIE,,--|,|-- NON-PLAYER ABILITY,--|,|--,NON-PLAYER REPUTATION,--|,|--,,,TECHNICAL ABILITY,,,,--|,|--,,,TECHNIQUE,,,--|,,|--,BUSINESS,--|,|--,,MENTAL,,--|,|--,,NOT IMPORTED,,--|'
    second_header = 'Mode (e),First Name,Second Name,Date of Birth,Nation,SecondNation,DeclaredNation,BirthTown,Classification,JobForClub,ClubContracted,ClubPlaying,DateJoinedClub,ContractExpiresClub,EstimatedWage,EstimatedWageWeekly,EstimatedValue,JobForNation,NationContracted,DateJoinedNation,ContractExpiresNation,InternationalApps,InternationalGoals,InternationalAssists,FirstNHLContract,StanleyCupsWon,New first name,New second name,New date of birth,Adaptability,Ambition,Determination,Loyalty,Pressure,Professionalism,Sportsmanship,Temperament,CurrentAbility,PotentialAbility,HomeReputation,CurrentReputation,WorldReputation,Goaltender,LeftDefense,RightDefense,LeftWing,Center,RightWing,DefensiveRole,OffensiveRole,Role,Hand,FavouriteNumber,SquadNumber,InternationalSquadNumber,HeightCm,WeightKg,JnrPreference,Aggression,Anticipation,Bravery,Consistency,Decisions,Dirtiness,Flair,ImportantMatches,Leadership,Morale,PassTendency,Teamwork,Creativity,WorkRate,Acceleration,Agility,Balance,Fighting,Hitting,InjuryProneness,NaturalFitness,Pace,Stamina,Strength,Agitation,Checking,Deflections,Deking,Faceoffs,Movement,OneOnOnes,Passing,Pokecheck,Positioning,Slapshot,Stickhandling,Versatility,Wristshot,Blocker,Glove,Rebounds,Recovery,Reflexes,CurrentAbility,PotentialAbility,HomeReputation,CurrentReputation,WorldReputation,PreferredJob,Attacking,Directness,FreeRoles,LineMatching,PenaltyKill,Physical,PowerPlay,CoachingGoaltenders,CoachingDefensemen,CoachingForwards,CoachingTechnique,Judgement,JudgingPotential,Tactics,Physiotherapy,Business,Patience,Resources,Discipline,Interference,ManHandling,Motivating,Youngsters,League contracted,League playing,Latest career history,NHL Draft Eligible,NHL Drafted'
    csvwriter.writerow(top_header.split(','))
    csvwriter.writerow(second_header.split(','))

    # Read input file
    csvreader = csv.reader(input_file, delimiter=',')
    for row in csvreader:
      # Skip header rows
      if row[0] != "":
        continue

      try:
        # Get the name, birth date, and birth place of a player or staff member
        first_name = row[1]
        last_name = row[2]
        dob = datetime.strptime(row[3], "%d.%m.%Y")

        # Search EP for the player or staff member
        # There may exist multiple players and/or staff members with the same name, so we check each's DOB to make sure it matches the input player or staff member
        request_link = f"https://api.eliteprospects.com/v1/search?q={first_name + ' ' + last_name}&apiKey={api_key}"
        response = requests.get(request_link)
        json = response.json()["data"]
        player_ids = [entry["id"] for entry in json["player"]]
        staff_ids = [entry["id"] for entry in json["staff"]]

        # For each matching player, denote whether any has played within the last 2 years
        # Also check if the date of birth for each matches the date of birth of the input staff or player
        recently_active = False
        for id in player_ids:
          # Check if DOB matches
          request_link = f"https://api.eliteprospects.com/v1/players/{id}?apiKey={api_key}"
          response = requests.get(request_link)
          player_data = response.json()["data"]
          date_of_birth = datetime.strptime(player_data["dateOfBirth"], '%Y-%m-%d')
          if date_of_birth == dob:
            # Get recent seasons
            request_link = f"https://api.eliteprospects.com/v1/players/{id}/stats?offset=0&limit=100&sort=season&apiKey={api_key}"
            response = requests.get(request_link)
            seasons = response.json()["data"]
            last_season = seasons[-1]["season"]["endYear"]
            if int(last_season) <= int(most_recent_player_year):
              recently_active = True
              break

        if not recently_active: # No reason to check staff if we have already found the right player
          for id in staff_ids:
            # Check if DOB matches
            request_link = f"https://api.eliteprospects.com/v1/staff/{id}?apiKey={api_key}"
            response = requests.get(request_link)
            player_data = response.json()["data"]
            date_of_birth = datetime.strptime(player_data["dateOfBirth"], '%Y-%m-%d')
            if date_of_birth == dob:
              # Get recent seasons
              request_link = f"https://api.eliteprospects.com/v1/staff/{id}/stats?offset=0&limit=100&sort=season&apiKey={api_key}"
              response = requests.get(request_link)
              seasons = response.json()["data"]
              last_season = seasons[-1]["season"]["endYear"]
              if int(last_season) <= int(most_recent_staff_year):
                recently_active = True
                break

        if recently_active:
          row[0] = "d"

        csvwriter.writerow(row)
      except Exception:
        row[0] = "e"
        csvwriter.writerow(row)